In [136]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer
# Librerías de visualización
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt


# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames
pd.set_option('display.max_colwidth', None) # para visualizar todo el contenido de los valores
#pd.set_option('display.max_rows', None) # Para visualizar todo el contenido de las filas

In [137]:
# Leemos el csv

datos = pd.read_csv('Datos/datos_empresa.csv', index_col=0)

### Transformación de Datos


In [138]:
# Borramos columnas que no necesitamos según el estudio EDA

columnas_borrar = ['employeecount','Over18','SameAsMonthlyIncome', 'Salary', 'RoleDepartament', 'NUMBERCHILDREN','employeenumber','StandardHours']

def borrar_colunas (datos, columnas):
    datos.drop( columns = columnas, inplace= True)

borrar_colunas(datos,columnas_borrar)

In [139]:
# Transformamos las cabeceras de las columnas para su mejor visulatización las convertimos todos a lower

new_columns = {column : column.lower() for column in datos}

new_columns

datos.rename(columns = new_columns, inplace= True)


#comprobamos que se ha realizado
datos.sample(2)

,age,attrition,businesstravel,dailyrate,department,distancefromhome,education,educationfield,environmentsatisfaction,gender,hourlyrate,jobinvolvement,joblevel,jobrole,jobsatisfaction,maritalstatus,monthlyincome,monthlyrate,numcompaniesworked,overtime,percentsalaryhike,performancerating,relationshipsatisfaction,stockoptionlevel,totalworkingyears,trainingtimeslastyear,worklifebalance,yearsatcompany,yearsincurrentrole,yearssincelastpromotion,yearswithcurrmanager,datebirth,remotework
79,fifty-eight,No,travel_rarely,"682,0$",NaN,-18,4,NaN,4,0,37,3,4,saLes ExECutiVE,3,Single,"13872,0",24409,0,NaN,13,"3,0",3,0,"38,0",1,"2,0",37,NaN,1,8,1965,1
1192,37,Yes,NaN,"625,0$",NaN,1,4,Life Sciences,1,0,46,2,3,SALeS execUtivE,3,Married,NaN,14922,5,NaN,11,"3,0",3,0,NaN,2,"1,0",14,NaN,11,7,1986,True


In [141]:
# Columna Age --- cambiamos los valores string a números

datos['age'].unique()

age = datos['age'].str.replace('fifty-eight', '58 ').str.replace('fifty-eight', '47 ').str.replace('thirty-six', '36').str.replace('fifty-five', '55').str.replace('fifty-two', '52').str.replace('thirty-one', '31').str.replace('twenty-six', '36').str.replace('thirty-seven', '37').str.replace('thirty-seven', '37').str.replace('thirty-two', '32').str.replace('twenty-four', '24').str.replace('forty-seven', '47').str.replace('thirty', '30')

datos['age'] = age

datos['age'] = datos['age']

datos['age'].unique()  



array(['51', '52', '42', '47', '46', '48', '59', '41', '56', '38', '55',
       '40', '58', '35', '45', '33', '36', '34', '53', '43', '60', '32',
       '37', '49', '39', '50', '44', '30', '58 ', '29', '31', '54', '57',
       '27', '28', '26', '25', '24', '23', '22', '21', '20', '19', '18'],
      dtype=object)

In [142]:
# Eliminamos los $ y ,0 con un .replace. También hemos pasado los nan y luego cambiamos el tipo a float

dailyrate = datos['dailyrate'].str.replace('$', '')
datos['dailyrate'] = dailyrate

datos['dailyrate'] = datos['dailyrate']

datos['dailyrate'].head(2)

0    684,0
1    699,0
Name: dailyrate, dtype: object

In [143]:
# Hemos cambiado el tipo de int a str y eliminado el menos de los negativos. Lo pasamos a int.

datos_distance = datos['distancefromhome'].astype(str)

distancefromhome = datos_distance.str.replace('-', '')

datos['distancefromhome'] = distancefromhome

datos['distancefromhome'] = datos['distancefromhome']

datos['distancefromhome'].unique()

array(['6', '1', '4', '2', '3', '22', '25', '9', '7', '23', '10', '12',
       '14', '13', '15', '8', '42', '28', '37', '5', '16', '35', '26',
       '24', '29', '17', '21', '18', '30', '27', '20', '31', '39', '11',
       '19', '33', '34', '46', '36', '45', '47', '32', '41', '49', '48',
       '38', '43', '40', '44'], dtype=object)

In [144]:
# Vamos a sustituir los 0 y 1 por la siniciales de su género y cambiarlo a tipo objeto.
dic_map = {0:'M', 1:'F'}

datos["gender"] = datos["gender"].map(dic_map)

datos['gender'].unique()

array(['M', 'F'], dtype=object)

In [145]:
# cambiar el Not Avaiable a NaN

datos['hourlyrate'] = datos['hourlyrate'].replace('Not Available', np.nan)

datos['hourlyrate'].unique() 

array(['51', '65', '58', '82', '45', '99', '91', '64', '55', '68', '49',
       '61', '79', '31', '69', '48', '80', '74', '98', '59', '33', '56',
       '66', '57', '53', '87', '81', '84', '32', '41', '92', '47', nan,
       '43', '86', '30', '42', '88', '96', '67', '62', '72', '78', '89',
       '52', '50', '90', '37', '94', '76', '60', '46', '83', '100', '40',
       '97', '54', '75', '39', '85', '63', '44', '93', '36', '35', '73',
       '71', '70', '38', '77', '95', '34'], dtype=object)

In [146]:
# Sustituimos los que están mal escritos para que sean iguales a los que si creando un diccionario previamente. Comprobar moda
replacements = {
    'divorced': 'Divorced',
    'Marreid': 'Married'
}

datos['maritalstatus'] = datos['maritalstatus'].replace(replacements)
datos['maritalstatus'].unique() 

array([nan, 'Married', 'Divorced', 'Single'], dtype=object)

In [147]:
# Cambiamos todos los datos a yes o NO depende la numeración

diccionario_mapa = {'Yes':'Yes', '1':'Yes', 'False':'No', '0':'No', 'True':'Yes'}

datos["remotework"] = datos["remotework"].map(diccionario_mapa)

datos['remotework'].unique()

array(['Yes', 'No'], dtype=object)

In [148]:
# Funcion para cambiar a float

cols_float = ['dailyrate', 'monthlyincome',  'performancerating', 'totalworkingyears', 'worklifebalance', 'yearsincurrentrole']

def cambiar_float(datos, cols):
    for col in cols:
        datos[col] = datos[col].apply(lambda dato: float(dato.replace(",", ".")) if isinstance(dato, str) else np.nan)

cambiar_float(datos, cols_float)

In [149]:
# Funcion para cambiar a int

cols_int = ['age', 'distancefromhome', 'hourlyrate']

def cambiar_float(datos, cols):
    for col in cols:
        datos[col] = datos[col].apply(lambda dato: int(dato) if isinstance(dato, str) else np.nan)

cambiar_float(datos, cols_int)

In [150]:
# Funcion para cambiar textos

cols_text = ['attrition', 'businesstravel', 'department', 'educationfield', 'gender', 'jobrole', 'maritalstatus', 'overtime', 'remotework']

def cambiar_texto(datos, cols):
    for col in cols:
        datos[col] = datos[col].str.strip().str.replace('-', '').str.replace('_', ' ').str.capitalize()
        

cambiar_texto(datos, cols_text)

In [151]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1614 entries, 0 to 1613
Data columns (total 33 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   age                       1614 non-null   int64  
 1   attrition                 1614 non-null   object 
 2   businesstravel            842 non-null    object 
 3   dailyrate                 1490 non-null   float64
 4   department                302 non-null    object 
 5   distancefromhome          1614 non-null   int64  
 6   education                 1614 non-null   int64  
 7   educationfield            869 non-null    object 
 8   environmentsatisfaction   1614 non-null   int64  
 9   gender                    1614 non-null   object 
 10  hourlyrate                1530 non-null   float64
 11  jobinvolvement            1614 non-null   int64  
 12  joblevel                  1614 non-null   int64  
 13  jobrole                   1614 non-null   object 
 14  jobsatisfacti

In [157]:
datos.sample(5)

,age,attrition,businesstravel,dailyrate,department,distancefromhome,education,educationfield,environmentsatisfaction,gender,hourlyrate,jobinvolvement,joblevel,jobrole,jobsatisfaction,maritalstatus,monthlyincome,monthlyrate,numcompaniesworked,overtime,percentsalaryhike,performancerating,relationshipsatisfaction,stockoptionlevel,totalworkingyears,trainingtimeslastyear,worklifebalance,yearsatcompany,yearsincurrentrole,yearssincelastpromotion,yearswithcurrmanager,datebirth,remotework
1189,42,No,Travel frequently,1271.0,Research & development,2,1,NaN,2,M,35.0,3,1,Research scientist,4,Single,NaN,9068,5,NaN,14,3.0,4,0,8.0,2,3.0,2,NaN,2,2,1981,Yes
799,34,No,Travel rarely,1346.0,NaN,38,2,NaN,2,M,93.0,3,1,Laboratory technician,4,Divorced,NaN,8758,0,NaN,11,3.0,3,1,3.0,2,3.0,2,NaN,1,2,1989,Yes
819,44,No,NaN,489.0,NaN,23,3,NaN,2,M,NaN,3,2,Laboratory technician,2,Married,NaN,25043,4,NaN,12,3.0,3,1,NaN,3,4.0,3,NaN,1,2,1979,Yes
1162,32,No,Travel rarely,529.0,NaN,2,3,Technical degree,4,M,78.0,3,1,Research scientist,1,NaN,NaN,11288,1,No,14,3.0,4,0,NaN,4,3.0,4,NaN,1,2,1991,Yes
1058,44,No,NaN,981.0,NaN,5,3,Life sciences,3,M,90.0,2,1,Laboratory technician,3,Single,3162.0,7973,3,NaN,14,3.0,4,0,7.0,5,3.0,5,NaN,0,3,1979,No


In [111]:
# Creamos copia de seguridad del archivo

datos.to_csv('Datos/datos_empresa_V.1.clean.csv')

### Gestion de Nulos

In [158]:
datos = pd.read_csv('Datos/datos_empresa_V.1.clean.csv', index_col=0)

In [164]:
# qué columnas tienen nulos

nulos_cat = datos[datos.columns[datos.isnull().any()]].select_dtypes(include = "O").columns
print("Las columnas categóricas que tienen nulos son : \n ")
print(nulos_cat)

Las columnas categóricas que tienen nulos son : 
 
Index(['businesstravel', 'department', 'educationfield', 'maritalstatus',
       'overtime'],
      dtype='object')


In [160]:
# Vemos la forma de estas columnas

for col in nulos_cat:
    print(f"La distribución de las categorías para la columna {col.upper()}")
    display(datos[col].value_counts() / datos.shape[0])
    print("........................")

La distribución de las categorías para la columna BUSINESSTRAVEL


businesstravel
Travel rarely        0.363073
Travel frequently    0.102230
Nontravel            0.056382
Name: count, dtype: float64

........................
La distribución de las categorías para la columna DEPARTMENT


department
Research & development    0.121437
Sales                     0.056382
Human resources           0.009294
Name: count, dtype: float64

........................
La distribución de las categorías para la columna EDUCATIONFIELD


...

........................
La distribución de las categorías para la columna MARITALSTATUS


maritalstatus
Married     0.271995
Single      0.201363
Divorced    0.123296
Name: count, dtype: float64

........................
La distribución de las categorías para la columna OVERTIME


overtime
No     0.422553
Yes    0.158612
Name: count, dtype: float64

........................


In [161]:
datos.isna().sum() / datos.shape[0] * 100

age                          0.000000
attrition                    0.000000
businesstravel              47.831475
dailyrate                    7.682776
department                  81.288724
distancefromhome             0.000000
education                    0.000000
educationfield              46.158612
environmentsatisfaction      0.000000
gender                       0.000000
hourlyrate                   5.204461
jobinvolvement               0.000000
joblevel                     0.000000
jobrole                      0.000000
jobsatisfaction              0.000000
maritalstatus               40.334572
monthlyincome               52.230483
monthlyrate                  0.000000
numcompaniesworked           0.000000
overtime                    41.883519
percentsalaryhike            0.000000
performancerating           12.081784
relationshipsatisfaction     0.000000
stockoptionlevel             0.000000
totalworkingyears           32.589839
trainingtimeslastyear        0.000000
worklifebala